In [1]:
import re
import os
import sys
import json
import gzip
import codecs

from time import sleep
from itertools import chain
from operator import itemgetter

from multiprocessing.dummy import Pool, Queue

import requests
import pandas as pd

from lxml import etree, html as lhtml
from tqdm.notebook import tqdm

In [2]:
PROXY = {'http': '3.141.186.75:3128', 'https': '3.141.186.75:3128', }
PROXY = {}

In [3]:
documents = []
page_num = 0
max_num_documents = 300

with tqdm(desc='Number of documents', total=max_num_documents) as pbar:
    while len(documents) < max_num_documents:
        page_num += 1
        page = requests.get('https://gg.deals/games/',
                            params={'sort': 'metascore', 'view': 'list', 'page': page_num},
                            proxies=PROXY)

        tree = lhtml.fromstring(page.content)

        documents_ = tree.xpath('//div[contains(@class, "game-item")]//a[@class="full-link"]/@href')
        documents_ = filter(lambda s: s.startswith('/game/'), documents_)
        documents_ = map(lambda s: 'https://gg.deals' + s, documents_)
        documents_ = list(documents_)

        documents.extend(documents_)
        pbar.update(len(documents_))

    documents = documents[:max_num_documents]

len(documents)

300

In [4]:
HEADERS_API = '''
accept: application/json, text/javascript, */*; q=0.01
accept-encoding: gzip, deflate, br
accept-language: ru
user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36
x-requested-with: XMLHttpRequest
'''

HEADERS_API = HEADERS_API.strip().split('\n')
HEADERS_API = [h.split(': ') for h in HEADERS_API]
HEADERS_API = dict(HEADERS_API)
HEADERS_API

{'accept': 'application/json, text/javascript, */*; q=0.01',
 'accept-encoding': 'gzip, deflate, br',
 'accept-language': 'ru',
 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
 'x-requested-with': 'XMLHttpRequest'}

In [5]:
HEADERS_DLC = '''
accept: */*
accept-encoding: gzip, deflate, br
accept-language: ru
content-type: application/x-www-form-urlencoded; charset=UTF-8
cookie: gg-session=35gfb7uu7eo7dsk4dr5qnk8svi; firstVisit=1651430665; gg_csrf=1b7c5bcfa3b8d404cc0286a30238be9266c8868as%3A88%3A%22a3Vqek9TQWZkX25HdEhrSXExU1EzSlBLdTFaWlQ2OEp91bl9fu3W6moq2YhoXf3Le0wuPTwDBxVxnPrUyEuaKQ%3D%3D%22%3B; ab_a0d29021=193021871; _hjFirstSeen=1; _hjIncludedInSessionSample=0; _hjSession_994870=eyJpZCI6ImM5NTJiMWI4LTY0MTYtNDA3Yy1hNDI4LTNkMTA4OWE0YmYyNSIsImNyZWF0ZWQiOjE2NTE0MzA2Njk0NzcsImluU2FtcGxlIjpmYWxzZX0=; _hjAbsoluteSessionInProgress=0; _ga=GA1.2.2114458876.1651430670; _gid=GA1.2.17703867.1651430670; _hjSessionUser_994870=eyJpZCI6IjIzM2JlMDdmLTIzNWYtNTY4Yy04NDFjLTNhNTYxNzZkMzNkMCIsImNyZWF0ZWQiOjE2NTE0MzA2Njk0NjYsImV4aXN0aW5nIjp0cnVlfQ==; _gat_UA-36788988-8=1; _hjIncludedInPageviewSample=1; _gat_testAB_a0d29021=1
user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36
x-requested-with: XMLHttpRequest
'''

HEADERS_DLC = HEADERS_DLC.strip().split('\n')
HEADERS_DLC = [h.split(': ') for h in HEADERS_DLC]
HEADERS_DLC = dict(HEADERS_DLC)
HEADERS_DLC

{'accept': '*/*',
 'accept-encoding': 'gzip, deflate, br',
 'accept-language': 'ru',
 'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
 'cookie': 'gg-session=35gfb7uu7eo7dsk4dr5qnk8svi; firstVisit=1651430665; gg_csrf=1b7c5bcfa3b8d404cc0286a30238be9266c8868as%3A88%3A%22a3Vqek9TQWZkX25HdEhrSXExU1EzSlBLdTFaWlQ2OEp91bl9fu3W6moq2YhoXf3Le0wuPTwDBxVxnPrUyEuaKQ%3D%3D%22%3B; ab_a0d29021=193021871; _hjFirstSeen=1; _hjIncludedInSessionSample=0; _hjSession_994870=eyJpZCI6ImM5NTJiMWI4LTY0MTYtNDA3Yy1hNDI4LTNkMTA4OWE0YmYyNSIsImNyZWF0ZWQiOjE2NTE0MzA2Njk0NzcsImluU2FtcGxlIjpmYWxzZX0=; _hjAbsoluteSessionInProgress=0; _ga=GA1.2.2114458876.1651430670; _gid=GA1.2.17703867.1651430670; _hjSessionUser_994870=eyJpZCI6IjIzM2JlMDdmLTIzNWYtNTY4Yy04NDFjLTNhNTYxNzZkMzNkMCIsImNyZWF0ZWQiOjE2NTE0MzA2Njk0NjYsImV4aXN0aW5nIjp0cnVlfQ==; _gat_UA-36788988-8=1; _hjIncludedInPageviewSample=1; _gat_testAB_a0d29021=1',
 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, l

In [6]:
def get_history(app_id, page_url):
    headers = HEADERS_API.copy()
    headers['referer'] = page_url
    
    page_url = f'https://gg.deals/ru/games/chartHistoricalData/{app_id}/'
    page = requests.get(page_url, params={'hideKeyshops': 0, 'showKeyshops': 1},
                        headers=headers, proxies=PROXY)
    
    if not page.ok:
        print('Failed to get page:', page_url)
        return False, []
    
    data = page.json()

    if data['code']:
        print('Error code for page:', page_url, file=sys.stderr)
        return False, []
    
    if 'chartData' not in data:
        print('No chartData for page:', page_url, file=sys.stderr)
        return False, []
    
    data = data['chartData']
    
    if 'deals' not in data:
        print('No deals for page:', page_url, file=sys.stderr)
        return False, []
    
    data = data['deals']

    history = [{'ts': r['x'] // 1000, 'price': r['y'], 'shop': r['shop']} for r in data]
    return True, history


def get_dlc_pack_info(game_id, page_url, type_):
    headers = HEADERS_DLC.copy()
    # headers['referer'] = page_url
    
    page = requests.post(
        f'https://gg.deals/ru/games/relations/{game_id}/', 
        params={'type': type_, 'offset': 0, 'hideKeyshops': 0, 'showKeyshops': 1},
        data='gg_csrf=a3Vqek9TQWZkX25HdEhrSXExU1EzSlBLdTFaWlQ2OEp91bl9fu3W6moq2YhoXf3Le0wuPTwDBxVxnPrUyEuaKQ==',
        headers=HEADERS_DLC,
    )
    
    tree = lhtml.fromstring(page.content)
    
    dlcs = tree.xpath('//a[@class="full-link"]/@href')
    dlcs = map(lambda s: 'https://gg.deals' + s, dlcs)
    dlcs = list(dlcs)
    
    return dlcs


def extract_page_info(tree):
    info = {}
    
    meta = tree.xpath('//div[@class="game-card"]')[0]
    
    name = meta.xpath('//ul[@class="breadcrumbs-list"]//a/span[@itemprop="name"]/text()')
    name = name[-1]
    info['name'] = name
    
    image = meta.xpath('//div[@class="game-info-image"]//img')[0].get('src')
    info['image'] = image
    
    release_date = meta.xpath('.//div[contains(@class, "game-info-details-section-release")]/p/text()')
    release_date = ''.join(release_date)
    info['release_date'] = release_date
    
    developer = meta.xpath('.//div[contains(@class, "game-info-details-section-developer")]/p/text()')
    developer = ''.join(developer)
    info['developer'] = developer
    
    metacritic_score = meta.xpath('.//a[contains(@class, "score-metascore")]//span[@class="overlay"]/text()')
    metacritic_score = ''.join(metacritic_score)
    if metacritic_score:
        info['metacritic_score'] = int(metacritic_score)
        
    user_score = meta.xpath('.//a[contains(@class, "score-userscore")]//span[@class="overlay"]/text()')
    user_score = ''.join(user_score)
    if user_score:
        info['user_score'] = float(user_score)
        
    reviews = meta.xpath('.//span[@class="reviews-label"]')
    if reviews:
        reviews = reviews[0]
        match = re.search(
            '(\d+)% of the (\d+(?:,\d+)*) user reviews for this game are positive.',
            reviews.attrib['title'],
        )
        
        info['review_label'] = reviews.text
        info['review_positive_pctg'] = int(match.group(1))
        info['review_count'] = int(re.sub('\D', '', match.group(2)))

    info['genres'] = meta.xpath('.//div[@id="game-info-genres"]//a[contains(@class, "tag")]/text()')
    info['tags'] = meta.xpath('.//div[@id="game-info-tags"]//a[contains(@class, "tag")]/text()')
    info['features'] = meta.xpath('.//div[@id="game-info-features"]//a[contains(@class, "tag")]/text()')
    info['platforms'] = meta.xpath('.//div[contains(@class, "game-info-details-section-platforms")]//svg/@title')
    
    card_actions = meta.xpath('//div[@class="game-info-actions game-single-actions"]')
    if card_actions:
        card_actions = card_actions[0]
        
        wishlist_cnt = card_actions.xpath('.//div[contains(@class, "wishlisted-game")]'
                                          '//span[@class="count"]/text()')
        wishlist_cnt = int(wishlist_cnt[0])
        info['wishlist_count'] = wishlist_cnt
        
        alert_cnt = card_actions.xpath('.//div[contains(@class, "alerted-game")]'
                                       '//span[@class="count"]/text()')
        alert_cnt = int(alert_cnt[0])
        info['alert_count'] = alert_cnt
        
        owners_cnt = card_actions.xpath('.//div[contains(@class, "owned-game")]'
                                        '//span[@class="count"]/text()')
        owners_cnt = int(owners_cnt[0])
        info['owners_count'] = owners_cnt
    
    market_url = meta.xpath('//a[@class="game-link-widget"]/@href')
    if market_url:
        market_url = market_url[0]
        market_url = requests.get(market_url, proxies=PROXY).url
        info['market_url'] = market_url
    
    return info


def process_page(page_url):
    info = {
        'url':    page_url,
        'status': False,
    }
    
    page = requests.get(page_url, proxies=PROXY)
    tree = lhtml.fromstring(page.content)
    app_id = int(tree.xpath('.//div[@data-game-id]/@data-game-id')[0])

    info.update(extract_page_info(tree))

    try:
        info['dlcs'] = get_dlc_pack_info(app_id, page_url, 'dlc')
    except:
        info['dlcs'] = []

    try:
        info['packs'] = get_dlc_pack_info(app_id, page_url, 'packs')
    except:
        info['packs'] = []
    
    is_ok, history = get_history(app_id, page_url)
    info['price_history'] = history
    info['status'] = is_ok
    
    return info

In [7]:
%%time

print(json.dumps(
    process_page('https://gg.deals/game/tricky-towers/'),
    indent=4,
))

{
    "url": "https://gg.deals/game/tricky-towers/",
    "status": true,
    "name": "Tricky Towers",
    "image": "https://img.gg.deals/00/ee/c8582c4fa19ba60498d50c1b6cfbc859afaa_307xt176.jpg",
    "release_date": "02 Aug 2016",
    "developer": " / ",
    "metacritic_score": 80,
    "user_score": 6.9,
    "review_label": "Very Positive",
    "review_positive_pctg": 90,
    "review_count": 8791,
    "genres": [
        "Casual",
        "Indie"
    ],
    "tags": [
        "Party Game",
        "Puzzle",
        "Physics",
        "Casual",
        "Building",
        "Multiplayer",
        "Funny",
        "Family Friendly",
        "Cartoony",
        "Arcade",
        "Local Multiplayer",
        "Singleplayer",
        "Online Co-Op",
        "Local Co-Op",
        "PvP",
        "4 Player Local",
        "Strategy",
        "Party",
        "Colorful",
        "Stylized",
        "Indie"
    ],
    "features": [
        "Single-player",
        "Online PvP",
        "Shared/Split

In [8]:
queue = Queue()

for url in documents:
    queue.put(url)

In [9]:
import os, shutil

if os.path.isdir('data'):
    shutil.rmtree('data')
os.mkdir('data')

In [10]:
def process_page_wrapper(i):
    with open('data/part_{:05d}.jsonl'.format(i), mode='w') as f_json:
        while not queue.empty():
            is_ok, reason = True, None
            url = queue.get()

            try:
                record = process_page(url)

                if record is not None:
                    record_str = json.dumps(record, ensure_ascii=False)
                    print(record_str, file=f_json)
                else:
                    is_ok, reason = False, 'Failed to get url'
            except Exception as e:
                is_ok, reason = False, e
                
                if isinstance(e, KeyboardInterrupt):
                    break
            finally:
                with lock:
                    if not is_ok:
                        print('Failed:', url, file=sys.stderr)
                        print('Reason:', reason, file=sys.stderr)
                    pbar.update(1)

In [ ]:
with Pool(processes=8) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))
    pool.close()
    pool.join()